# A simple Python plugin system

(Inspired by the plugin system from the Bottle web development library: http://bottlepy.org)

#### Define some global state

In [71]:
state={}

A plugin is a function that wraps another function

In [36]:
def plugin1(callback):
    def wrapper(*args,**kwargs):
        print("open db connection")
        state['db']=True
        output =  callback(*args,**kwargs)
        print("close db connection")
        del state['db'] 
        return output
    return wrapper

In [37]:
def plugin2(callback):
    def wrapper(*args,**kwargs):
        print("load session object")
        assert state['db']
        state['session'] = True
        output =  callback(*args,**kwargs)
        print("save session object")
        del state['session'] 
        return output
    return wrapper

In [38]:
def plugin3(callback):
    def wrapper(*args,**kwargs):
        print("check authentication")
        assert state['session']
        state['user'] = "a user object"
        output =  callback(*args,**kwargs)
        del state['user']
        return output
    return wrapper

In [74]:
def myfunc():
    'actually do some business logic'
    user = state['user']
    print("The user is",user)

If we call `myfunc()` without plugins, it fails to work, because the relevant global state has not yet been set up.

In [75]:
myfunc()

KeyError: 'user'

So we create a mechanism for installing plugins and calling them sequentially on a provided function

In [76]:
plugin_stack=[]

def install(plugin):
    plugin_stack.append(plugin)
    
def call_with_plugins(func):
    
    for plugin in reversed(plugin_stack):
        func = plugin(func)
        
    return func()


In [77]:
install(plugin1)
install(plugin2)
install(plugin3)

In [78]:
call_with_plugins(myfunc)

open db connection
load session object
check authentication
The user is a user object
save session object
close db connection
